In [2]:
!pip install face-alignment

  Created wheel for face-alignment: filename=face_alignment-1.3.4-py2.py3-none-any.whl size=27859 sha256=9d6536c635595e6b0badecc496ffb670929a41e6c036d3202902c144465e03fe
  Stored in directory: /root/.cache/pip/wheels/a1/4d/d9/fa80a2341395ce73765c09eef97b262a048b2a763c9b689c2c
Successfully built face-alignment


In [7]:
%matplotlib inline
import face_alignment
from skimage import io,transform,data,color,feature
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os
import skimage.io as io
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score , GridSearchCV
from sklearn.svm import LinearSVC
import numpy as np
from numba import jit, cuda
import pickle
import timeit
from os import listdir
from matplotlib import image
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F

from __future__ import print_function
import argparse
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
import dlib
from imutils.face_utils import FaceAligner

In [8]:
import librosa
import soundfile
import glob
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [10]:
# put here the path to the mode
model = pickle.load(open("faceDetection2.sav", 'rb'))
print(model)
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=True)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)


Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-cd938726ad.zip" to /root/.cache/torch/hub/checkpoints/2DFAN4-cd938726ad.zip


In [11]:
#load the model 
voice_model = pickle.load(open('voice_model.pkl', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neural_network.multilayer_perceptron module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neural_network. Anything that cannot be imported from sklearn.neural_network is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.21.2 when

In [12]:
def getFaceAlign(img):
  io.imsave('testt.jpg',img)
  preds = fa.get_landmarks(img)
  if preds == None:
    print('Warning: No faces were detected.')
    return None,0
  if preds!=None:
    mnXY= np.min(np.min(preds, axis=1), axis=0)
    mxXY= np.max(np.max(preds, axis=1), axis=0)
    mxY=int(mxXY[0]+2)
    mnX=int(mnXY[1])-8
    mxX=int(mxXY[1])
    if mnX < 0:
       mnX=0
    if int(mxXY[0]+2) >= img.shape[1]:
      mxY=img.shape[1]-1
    if (img.shape[0]-2-mnX) <= 40:
      return img[mnX:img.shape[0]-2,int(mnXY[0])-2:mxY],1
    else:
      return img[mnX:mnX+40,int(mnXY[0])-2:mxY],1

In [13]:
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    pick = []
    x1 = boxes[:,2]
    y1 = boxes[:,0]
    x2 = boxes[:,3]
    y2 = boxes[:,1]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    return boxes[pick].astype("int")

In [14]:
def find(image):
    boxes=[[i,i+k,j,int(j+k)] for k in [35,45,55] for i in range(0,image.shape[0]-k,5) for j in range(0,image.shape[1]-k,5) if (model.predict([feature.hog(transform.resize(image[i:i+k,j:int(j+k)],(48,48)))])[0] == 1)]
    return boxes

In [15]:

import sys

from torch.nn.modules.activation import ReLU
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def conv_bn_relu(in_channels, out_channels, kernel_size=3, stride=1, padding=0):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )


def SeparableConv2D(in_channels, out_channels, kernel=3):
    return nn.Sequential(
        nn.Conv2d(in_channels, in_channels, kernel_size=kernel, stride=1, groups=in_channels,padding=1, bias=False),
        nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
    )

class ResidualXceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel=3):
        super(ResidualXceptionBlock, self).__init__()
        global device

        self.depthwise_conv1 = SeparableConv2D(in_channels, out_channels, kernel).to(device)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU(inplace=True)

        self.depthwise_conv2 = SeparableConv2D(out_channels, out_channels, kernel).to(device)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # self.padd = nn.ZeroPad2d(22)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.residual_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2, padding=22, bias=False)
        self.residual_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.residual_bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        # residual branch
        residual = self.residual_conv(x)
        residual = self.residual_bn(residual)
        
        # print('input',x.shape)
        # feature extraction branch
        x = self.depthwise_conv1(x)
        # print('conv1',x.shape)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.depthwise_conv2(x)
        x = self.bn2(x)
        # print('conv2',x.shape)

        # x = self.padd(x)
        x = self.maxpool(x)
        # print(x[:,:, 11:22, 11:22])
        # print('max_pooling',x.shape)
        # print('res',residual.shape)
        return x + residual

class Mini_Xception(nn.Module):
    def __init__(self):
        super(Mini_Xception, self).__init__()
        self.conv1 = conv_bn_relu(1, 8, kernel_size=3, stride=1, padding=0)
        self.conv2 = conv_bn_relu(8, 8, kernel_size=3, stride=1, padding=0)
        self.residual_blocks = nn.ModuleList([
            ResidualXceptionBlock(8 , 16).to(device),
            ResidualXceptionBlock(16, 32).to(device),
            ResidualXceptionBlock(32, 64).to(device),
            ResidualXceptionBlock(64, 128).to(device)            
        ])
        self.conv3 = nn.Conv2d(128, 7, kernel_size=3, stride=1, padding=1)

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.conv2(x)

        for block in self.residual_blocks:
            x = block(x)
            # print('ith block', x.shape, block.device)

        # print('blocks:',x.shape)
        x = self.conv3(x)
        # print('conv3',x.shape)
        x = self.global_avg_pool(x)
        # # x = self.softmax(x)

        return x

def get_label_emotion(label : int) -> str:
    label_emotion_map = { 
        0: 'Angry',
        1: 'Disgust', 
        2: 'Fear', 
        3: 'Happy', 
        4: 'Sad', 
        5: 'Surprise', 
        6: 'Neutral'        
    }
    return label_emotion_map[label]


def pr(img):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  mini_xception = Mini_Xception().to(device)
  mini_xception.eval()

  # Load model
  # put here the path to the model
  checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/weights_epoch_75.pth.tar')
  mini_xception.load_state_dict(checkpoint['mini_xception'])

    
  input_face = cv2.resize(img, (48,48))
  input_face = cv2.equalizeHist(input_face)
  input_face = transforms.ToTensor()(input_face).to(device)
  input_face = torch.unsqueeze(input_face, 0)

  with torch.no_grad():
      input_face = input_face.to(device)
      emotion = mini_xception(input_face)
      # print(f'\ntime={(time.time()-t) * 1000 } ms')

      torch.set_printoptions(precision=6)
      softmax = torch.nn.Softmax()
      emotions_soft = softmax(emotion.squeeze()).reshape(-1,1).cpu().detach().numpy()
      emotions_soft = np.round(emotions_soft, 3)
      for i, em in enumerate(emotions_soft):
          em = round(em.item(),3)
          # print(f'{get_label_emotion(i)} : {em}')

      emotion = torch.argmax(emotion)                
      percentage = round(emotions_soft[emotion].item(), 2)
      emotion = emotion.squeeze().cpu().detach().item()
      emotion = get_label_emotion(emotion)
      return emotion

# Voice analysis

In [16]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(X, mfcc, chroma, mel,sample_rate):   
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        stft=np.abs(librosa.stft(X))
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [17]:
def predict_voice(file):
    x=[]
    y=[]
    y, s = librosa.load(file) # Downsample 44.1kHz to 8kHz
    feature=extract_feature(X=y, mfcc=True, chroma=True, mel=True,sample_rate=s)
    x.append(feature)
    y_pred=voice_model.predict(x)
    emotion = y_pred[0]
    return emotion

In [18]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, request, jsonify, render_template
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
#this contains the path of folder that to store the images in
app.config["IMAGE_UPLOADS"] = "/content"
app.config["AUDIO_UPLOADS"]="/content"
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/') 
def home():
    return render_template('index.html')

@app.route("/predict", methods=['POST'])
def predict():
    url = request.method
    image = request.files["image"]
    
    #the name of image file
    imgName = image.filename
    #save the image in colab directory /content 
    image.save(os.path.join(app.config["IMAGE_UPLOADS"], image.filename))
    frameTemp = cv2.imread(imgName,0)
    frameTemp = cv2.resize(frameTemp, (100, 100))
    rects = find(frameTemp)
    rects = np.array(rects)
    rects = non_max_suppression_fast(rects, 0.2)
    output=""
    for rect in rects:
        res,ret=getFaceAlign(frameTemp[rect[0]:rect[1],rect[2]:rect[3]])
        if ret==1:
          print(rect)
          io.imshow(res)
          print(pr(res))
          output+= pr(res)+" "
    
    #return the emotion of the face's image to the html 
    return output

@app.route('/predictVoice',methods=['POST'])
def predictVoice():
    audio = request.files["audio"]
    print(audio)
    audiofile=audio.filename
    #save the audio in colab directory /content 
    audio.save(os.path.join(app.config["AUDIO_UPLOADS"], audio.filename))
    #dir of audio is in the current directory 
    emotion = predict_voice("/content/"+audiofile)
    return emotion
app.run()